## Visualize Data

In [8]:
from pathlib import Path
import numpy as np

data_path = Path("data/mariel_chunli.npy")
data_raw = np.load(data_path)

def inspect_data(data):
    print("shape", data.shape)
    print("dtype", data.dtype)
    print("max", data.max())
    print("min", data.min())
    print("1q", np.percentile(data, 25))
    print("2q", np.percentile(data, 50))
    print("3q", np.percentile(data, 75))
    print("mean", data.mean())
    print("std", data.std())

inspect_data(data_raw)

shape (55, 4866, 3)
dtype float64
max 1.9273922443389893
min -6.332875728607178
1q -1.006934016942978
2q -0.4554957449436188
3q -0.042966997250914574
mean -0.6010307200624114
std 0.8776853231783209


In [9]:
# Taken from https://github.com/mariel-pettee/choreo-graph/blob/main/functions/load_data.py

all_point_labels = ['ARIEL', 'C7', 'CLAV', 'LANK', 'LBHD', 'LBSH', 'LBWT', 'LELB', 'LFHD', 'LFRM', 'LFSH', 'LFWT', 'LHEL', 'LIEL', 'LIHAND', 'LIWR', 'LKNE', 'LKNI', 'LMT1', 'LMT5', 'LOHAND', 'LOWR', 'LSHN', 'LTHI', 'LTOE', 'LUPA', 'LabelingHips', 'MBWT', 'MFWT', 'RANK', 'RBHD', 'RBSH', 'RBWT', 'RELB', 'RFHD', 'RFRM', 'RFSH', 'RFWT', 'RHEL', 'RIEL', 'RIHAND', 'RIWR', 'RKNE', 'RKNI', 'RMT1', 'RMT5', 'ROHAND', 'ROWR', 'RSHN', 'RTHI', 'RTOE', 'RUPA', 'STRN', 'SolvingHips', 'T10']    
bad_labels = ['SolvingHips', 'LabelingHips']
point_labels = [label for label in all_point_labels if label not in bad_labels]

skeleton_lines = [
#     ( (start group), (end group) ),
    ('LHEL', 'LTOE',), # toe to heel
    ('RHEL', 'RTOE',),
    ('LMT1', 'LMT5',), # horizontal line across foot
    ('RMT1', 'RMT5',),   
    ('LHEL', 'LMT1',), # heel to sides of feet
    ('LHEL', 'LMT5',),
    ('RHEL', 'RMT1',),
    ('RHEL', 'RMT5',),
    ('LTOE', 'LMT1',), # toe to sides of feet
    ('LTOE', 'LMT5',),
    ('RTOE', 'RMT1',),
    ('RTOE', 'RMT5',),
    ('LKNE', 'LHEL',), # heel to knee
    ('RKNE', 'RHEL',),
    ('LFWT', 'RBWT',), # connect pelvis
    ('RFWT', 'LBWT',), 
    ('LFWT', 'RFWT',), 
    ('LBWT', 'RBWT',),
    ('LFWT', 'LBWT',), 
    ('RFWT', 'RBWT',), 
    ('LFWT', 'LTHI',), # pelvis to thighs
    ('RFWT', 'RTHI',), 
    ('LBWT', 'LTHI',), 
    ('RBWT', 'RTHI',), 
    ('LKNE', 'LTHI',), 
    ('RKNE', 'RTHI',), 
    ('CLAV', 'LFSH',), # clavicle to shoulders
    ('CLAV', 'RFSH',), 
    ('STRN', 'LFSH',), # sternum & T10 (back sternum) to shoulders
    ('STRN', 'RFSH',), 
    ('T10', 'LFSH',), 
    ('T10', 'RFSH',), 
    ('C7', 'LBSH',), # back clavicle to back shoulders
    ('C7', 'RBSH',), 
    ('LFSH', 'LBSH',), # front shoulders to back shoulders
    ('RFSH', 'RBSH',), 
    ('LFSH', 'RBSH',),
    ('RFSH', 'LBSH',),
    ('LFSH', 'LUPA',), # shoulders to upper arms
    ('RFSH', 'RUPA',), 
    ('LBSH', 'LUPA',), 
    ('RBSH', 'RUPA',), 
    ('LIWR', 'LIHAND',), # wrist to hand
    ('RIWR', 'RIHAND',),
    ('LOWR', 'LOHAND',), 
    ('ROWR', 'ROHAND',),
    ('LIWR', 'LOWR',), # across the wrist 
    ('RIWR', 'ROWR',), 
    ('LIHAND', 'LOHAND',), # across the palm 
    ('RIHAND', 'ROHAND',), 
    ('LFHD', 'LBHD',), # draw lines around circumference of the head
    ('LBHD', 'RBHD',),
    ('RBHD', 'RFHD',),
    ('RFHD', 'LFHD',),
    ('LFHD', 'ARIEL'), # connect circumference points to top of head
    ('LBHD', 'ARIEL'),
    ('RBHD', 'ARIEL'),
    ('RFHD', 'ARIEL'),
]

point_idxs = {label: i for i, label in enumerate(point_labels)}
edges = [(point_idxs[start], point_idxs[end]) for start, end in skeleton_lines]
edges = np.array(edges)
edges.shape

(58, 2)

In [10]:
len(point_labels)

53

First dimension is the edge group.
Second is the frames for a given clip.
Third is the XYZ coordinates in 3D space. 

We know the edge groups from [Pettee's previous project](https://github.com/mariel-pettee/choreography/blob/master/functions/functions.py), and we also know that groups 27 and 54 are bad edge groups. 

We reshape and mask those out. Additionally, we preprocess the data to be in the range -1 to 1.

In [11]:
def preprocess_data(data, normalize=True):
    bad_groups = [i for i, group in enumerate(all_point_labels) if group in bad_labels]
    group_mask = np.ones(data.shape[0], dtype=bool)
    group_mask[bad_groups] = False
    data = data[group_mask]

    data = data.swapaxes(0, 1)

    if normalize:
        min_val = data.min()
        max_val = data.max()
        data = (data - min_val) / (max_val - min_val) * 2 - 1

    data[:, :,  2] *= -1        # invert z axis

    return data

data = preprocess_data(data_raw, normalize=True)
inspect_data(data)

shape (4866, 53, 3)
dtype float64
max 1.0
min -1.0
1q -0.2738534668068461
2q 0.3330744535997223
3q 0.5042046208517127
mean 0.15681260326528199
std 0.4136803405246239


In [25]:
%matplotlib QtAgg

from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

class FigureAnimation:
    def __init__(self, data, edges):
        self.data = data
        self.edges = edges

        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(projection='3d')

        self.ax.set_xlim(data[:, :, 0].min(), data[:, :, 0].max())
        self.ax.set_ylim(data[:, :, 1].min(), data[:, :, 1].max())
        self.ax.set_zlim(data[:, :, 2].min(), data[:, :, 2].max())

        self.scatter_plot = None
        self.lineplots = None

    def start(self, start_frame: int = 0, end_frame: int = -1, framerate: int = 20):
        if end_frame == -1:
            end_frame = self.data.shape[0]

        def setup():
            snapshot = self.data[start_frame]
            self.scatter_plot = self.ax.scatter(snapshot[:, 0], snapshot[:, 1], snapshot[:, 2])

            start_edges, end_edges = self.edges[:, 0], self.edges[:, 1]
            lines = np.stack([snapshot[start_edges], snapshot[end_edges]], axis=1)
            
            self.lineplots = [self.ax.plot(line[:, 0], line[:, 1], line[:, 2], color='black')[0] for line in lines]

            return self.scatter_plot, *self.lineplots

        def update(frame):
            snapshot = self.data[int(frame)]
            self.scatter_plot._offsets3d = (snapshot[:, 0], snapshot[:, 1], snapshot[:, 2])

            start_edges, end_edges = self.edges[:, 0], self.edges[:, 1]
            lines = np.stack([snapshot[start_edges], snapshot[end_edges]], axis=1)
            for lineplot, line in zip(self.lineplots, lines):
                lineplot.set_data(line[:, 0], line[:, 1])
                lineplot.set_3d_properties(line[:, 2]) 

            self.fig.canvas.draw_idle()

            return self.scatter_plot, *self.lineplots

        self.ani = FuncAnimation(self.fig, update, init_func=setup, frames=range(start_frame, end_frame), interval=1000/framerate, blit=False)
        self.fig.show()


fig_anim = FigureAnimation(data, edges)
fig_anim.start(start_frame=150, framerate=60)